# 📝 Módulo 8 — Projeto Final: Enviar relatórios por e-mail

## 🎯 Objetivos
- Automatizar o envio de relatórios de vendas por e-mail.
- Gerar um resumo (KPIs) a partir de um CSV e enviar em formato **HTML** (no corpo) e **anexo** (CSV/HTML/PDF opcional).
- Preparar o script para agendamento (cron/Task Scheduler).

---

## ✅ Pré-requisitos
- **Python 3.10+**
- Conta de e-mail com SMTP habilitado (ex.: Gmail com **App Password**).
- Biblioteca(s) Python listadas em `requirements.txt`.

> 🔐 **Segurança**: nunca salve senhas no código. Use variáveis de ambiente (arquivo `.env`).

---

## 🗂️ Estrutura sugerida de pastas

```bash
projeto-relatorio-email/
├─ data/
│ └─ vendas.csv
├─ templates/
│ └─ email.html.j2
├─ reports/
│ └─ (saídas geradas: .html/.pdf)
├─ .env
├─ requirements.txt
└─ send_report.p
```



---

## 📦 requirements.txt
```txt
selenium>=4.23
webdriver-manager>=4.0
python-dotenv>=1.0
pandas>=2.2
jinja2>=3.1

```
---
## 🔐 .env (exemplo)

# LOGIN (usando web)
EMAIL_USER=seuemail@gmail.com
EMAIL_PASSWORD=sua_senha_ou_app_password

# DESTINATÁRIOS
EMAIL_TO=gestor@empresa.com,financeiro@empresa.com
EMAIL_CC=
EMAIL_BCC=

# ASSUNTO E IDENTIFICAÇÃO
EMAIL_SUBJECT=[Relatório Diário] Vendas
ORGANIZACAO=Minha Empresa LTDA

# CAMINHOS
CSV_PATH=./data/vendas.csv
TEMPLATE_PATH=./templates/email.html.j2
REPORTS_DIR=./reports

# NAVEGADOR (opcional)
USER_DATA_DIR=./chrome_profile  # reutiliza sessão (evita login todo dia)
HEADLESS=false                  # "true" para rodar sem abrir janela (pode falhar p/ logins)

---
# CSV de exemplo (data/vendas.csv)
```txt

data,pedido_id,cliente,produto,quantidade,preco
2025-09-20,1001,Ana,Sabão,3,8.90
2025-09-20,1002,Bruno,Detergente,2,6.50
2025-09-21,1003,Carlos,Água Sanitária,5,7.90
2025-09-21,1004,Débora,Esponja,10,2.50
2025-09-21,1005,Enzo,Sabão,1,8.90
```
---

# 🧩 Template HTML (templates/email.html.j2)

```html
<!doctype html>
<html>
  <head>
    <meta charset="utf-8"/>
    <title>Relatório de Vendas</title>
    <style>
      body { font-family: Arial, Helvetica, sans-serif; }
      .kpi { display:inline-block; margin:8px 16px; padding:8px 12px; border:1px solid #ddd; border-radius:8px; }
      table { border-collapse: collapse; width: 100%; margin-top:12px; }
      th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
      th { background: #f2f2f2; }
      .small { color:#666; font-size:12px; }
    </style>
  </head>
  <body>
    <h2>Relatório de Vendas — {{ data_referencia }}</h2>
    <p class="small">Organização: {{ organizacao }}</p>

    <div>
      <div class="kpi"><b>Receita Total</b><br>R$ {{ receita_total }}</div>
      <div class="kpi"><b>Qtde. Itens</b><br>{{ itens_total }}</div>
      <div class="kpi"><b>Tíquete Médio</b><br>R$ {{ ticket_medio }}</div>
      <div class="kpi"><b>Pedidos</b><br>{{ pedidos_total }}</div>
      <div class="kpi"><b>Mais Vendido</b><br>{{ produto_top }}</div>
    </div>

    <h3>Top Produtos</h3>
    <table>
      <thead>
        <tr><th>Produto</th><th>Quantidade</th><th>Receita (R$)</th></tr>
      </thead>
      <tbody>
        {% for row in top_produtos %}
        <tr>
          <td>{{ row.produto }}</td>
          <td>{{ row.quantidade }}</td>
          <td>{{ row.receita }}</td>
        </tr>
        {% endfor %}
      </tbody>
    </table>

    <p class="small">Relatório gerado automaticamente e enviado via automação de navegador.</p>
  </body>
</html>
```



# 🐍 Script principal (send_report_selenium.py)

```python
import os
import time
from datetime import datetime
from pathlib import Path

import pandas as pd
from jinja2 import Template
from dotenv import load_dotenv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# ============== Config & Util ==============

def load_cfg():
    load_dotenv()
    def env_bool(key, default="false"):
        return os.getenv(key, default).strip().lower() in {"1","true","yes","on"}
    return {
        "EMAIL_USER": os.getenv("EMAIL_USER"),
        "EMAIL_PASSWORD": os.getenv("EMAIL_PASSWORD"),
        "EMAIL_TO": [e.strip() for e in os.getenv("EMAIL_TO","").split(",") if e.strip()],
        "EMAIL_CC": [e.strip() for e in os.getenv("EMAIL_CC","").split(",") if e.strip()],
        "EMAIL_BCC": [e.strip() for e in os.getenv("EMAIL_BCC","").split(",") if e.strip()],
        "EMAIL_SUBJECT": os.getenv("EMAIL_SUBJECT","[Relatório] Vendas"),
        "ORGANIZACAO": os.getenv("ORGANIZACAO","Minha Empresa"),
        "CSV_PATH": os.getenv("CSV_PATH","./data/vendas.csv"),
        "TEMPLATE_PATH": os.getenv("TEMPLATE_PATH","./templates/email.html.j2"),
        "REPORTS_DIR": os.getenv("REPORTS_DIR","./reports"),
        "USER_DATA_DIR": os.getenv("USER_DATA_DIR",""),
        "HEADLESS": env_bool("HEADLESS","false"),
    }

def ensure_paths(cfg):
    Path(cfg["REPORTS_DIR"]).mkdir(parents=True, exist_ok=True)
    if not Path(cfg["CSV_PATH"]).exists():
        raise FileNotFoundError(f"CSV não encontrado: {cfg['CSV_PATH']}")
    if not Path(cfg["TEMPLATE_PATH"]).exists():
        raise FileNotFoundError(f"Template não encontrado: {cfg['TEMPLATE_PATH']}")

def build_driver(cfg):
    chrome_opts = webdriver.ChromeOptions()
    if cfg["USER_DATA_DIR"]:
        Path(cfg["USER_DATA_DIR"]).mkdir(parents=True, exist_ok=True)
        chrome_opts.add_argument(f"--user-data-dir={Path(cfg['USER_DATA_DIR']).resolve()}")
    if cfg["HEADLESS"]:
        chrome_opts.add_argument("--headless=new")
    chrome_opts.add_argument("--disable-gpu")
    chrome_opts.add_argument("--window-size=1366,900")
    chrome_opts.add_argument("--no-sandbox")
    chrome_opts.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_opts)
    driver.implicitly_wait(2)
    return driver

# ============== Relatório (pandas + jinja2) ==============

def carregar_df(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path, parse_dates=["data"])
    df["quantidade"] = pd.to_numeric(df["quantidade"], errors="coerce").fillna(0).astype(int)
    df["preco"] = pd.to_numeric(df["preco"], errors="coerce").fillna(0.0)
    df["receita"] = df["quantidade"] * df["preco"]
    return df

def calcular_kpis(df: pd.DataFrame) -> dict:
    receita_total = float(df["receita"].sum())
    itens_total = int(df["quantidade"].sum())
    pedidos_total = df["pedido_id"].nunique()
    ticket_medio = float(receita_total / max(1, pedidos_total))
    top = (
        df.groupby("produto", as_index=False)
          .agg(quantidade=("quantidade","sum"), receita=("receita","sum"))
          .sort_values(["quantidade","receita"], ascending=False)
          .head(5)
    )
    top_fmt = [{"produto": r["produto"], "quantidade": int(r["quantidade"]), "receita": f"{r['receita']:.2f}"} for _, r in top.iterrows()]
    return {
        "receita_total": f"{receita_total:.2f}",
        "itens_total": itens_total,
        "ticket_medio": f"{ticket_medio:.2f}",
        "pedidos_total": int(pedidos_total),
        "produto_top": top_fmt[0]["produto"] if top_fmt else "-",
        "top_produtos": top_fmt,
    }

def render_html(template_path: str, ctx: dict) -> str:
    tpl = Template(Path(template_path).read_text(encoding="utf-8"))
    return tpl.render(**ctx)

def salvar_html(html: str, reports_dir: str, prefixo: str="relatorio-vendas") -> Path:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    out = Path(reports_dir) / f"{prefixo}-{ts}.html"
    out.write_text(html, encoding="utf-8")
    return out

# ============== Selenium helpers (Gmail UI muda com frequência) ==============

def wait_css(driver, css, timeout=20):
    return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def wait_click(driver, locator, timeout=20):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))

def try_type(driver, candidates, text):
    for by, sel in candidates:
        try:
            el = WebDriverWait(driver, 5).until(EC.presence_of_element_located((by, sel)))
            el.clear()
            el.send_keys(text)
            return True
        except Exception:
            continue
    return False

def set_body_html_via_js(driver, html):
    # Corpo do e-mail é um DIV contenteditable; labels variam por idioma.
    candidates = [
        (By.CSS_SELECTOR, 'div[aria-label="Message Body"]'),
        (By.CSS_SELECTOR, 'div[aria-label="Corpo da mensagem"]'),
        (By.CSS_SELECTOR, 'div[role="textbox"][g_editable="true"]')
    ]
    for by, sel in candidates:
        try:
            el = WebDriverWait(driver, 5).until(EC.presence_of_element_located((by, sel)))
            driver.execute_script("arguments[0].innerHTML = arguments[1];", el, html)
            return True
        except Exception:
            continue
    return False

def anexar_arquivo_compose(driver, file_path: Path):
    # Abra o seletor de anexos para garantir input[type=file] visível
    try:
        # botão de anexar (tooltip muda por idioma)
        btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(
            (By.XPATH, '//div[contains(@data-tooltip,"Anexar") or contains(@data-tooltip,"Attach") or contains(@aria-label,"Anexar") or contains(@aria-label,"Attach")]')
        ))
        btn.click()
        time.sleep(1)
    except Exception:
        pass
    # procure o input file na janela de composição
    inputs = driver.find_elements(By.CSS_SELECTOR, 'input[type="file"]')
    if not inputs:
        # às vezes o input está fora do compose, mas ainda funciona
        inputs = driver.find_elements(By.XPATH, '//input[@type="file"]')
    if not inputs:
        raise RuntimeError("input[type=file] não encontrado para anexar.")
    inputs[0].send_keys(str(file_path.resolve()))
    # aguarda upload terminar (aparece uma barra/label de upload)
    time.sleep(3)

# ============== Fluxo principal ==============

def enviar_email_via_gmail(cfg, html_body: str, anexar: Path | None):
    driver = build_driver(cfg)
    wait = WebDriverWait(driver, 25)
    try:
        driver.get("https://mail.google.com/")
        # Se não estiver logado, efetua login básico (pode exigir 2FA manual)
        time.sleep(2)
        if "ServiceLogin" in driver.current_url or "signin" in driver.current_url:
            # e-mail
            try_type(driver, [
                (By.ID, "identifierId"),
                (By.NAME, "identifier")
            ], cfg["EMAIL_USER"])
            try:
                wait_click(driver, (By.ID, "identifierNext")).click()
            except Exception:
                pass
            # senha
            try:
                pwd = wait.until(EC.presence_of_element_located((By.NAME, "Passwd")))
                pwd.send_keys(cfg["EMAIL_PASSWORD"])
                wait_click(driver, (By.ID, "passwordNext")).click()
            except Exception:
                # Pode ser 2FA; aguarda intervenção manual
                print("Aguardando autenticação/2FA manual...")
            # dá tempo para concluir login
            time.sleep(10)

        # Garante inbox carregada
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))

        # Clicar em "Escrever" (Compose)
        try:
            wait_click(driver, (By.CSS_SELECTOR, 'div[gh="cm"]')).click()
        except Exception:
            # fallback por tooltip/aria
            wait_click(driver, (By.XPATH, '//div[contains(@data-tooltip,"Escrever") or contains(@data-tooltip,"Compose")]')).click()

        # Preencher "Para"
        to_text = ", ".join(cfg["EMAIL_TO"] + cfg["EMAIL_CC"] + cfg["EMAIL_BCC"])
        ok_to = try_type(driver, [
            (By.NAME, "to"),
            (By.CSS_SELECTOR, 'textarea[name="to"]'),
            (By.XPATH, '//div[@aria-label="Para" or @aria-label="To"]//input')
        ], to_text)
        if not ok_to:
            raise RuntimeError("Campo 'Para' não localizado.")

        # Assunto
        ok_subject = try_type(driver, [
            (By.NAME, "subjectbox"),
            (By.CSS_SELECTOR, 'input[name="subjectbox"]')
        ], cfg["EMAIL_SUBJECT"])
        if not ok_subject:
            raise RuntimeError("Campo 'Assunto' não localizado.")

        # Corpo (HTML via JS para preservar formatação)
        ok_body = set_body_html_via_js(driver, html_body)
        if not ok_body:
            # fallback: colar como texto se não conseguir setar innerHTML
            body = wait_css(driver, 'div[role="textbox"][g_editable="true"]')
            body.click()
            body.send_keys("Seu cliente não aceitou HTML. Consulte o anexo do relatório.")
        
        # Anexo (HTML salvo)
        if anexar:
            anexar_arquivo_compose(driver, anexar)

        # Enviar
        try:
            send_btn = wait_click(driver, (By.XPATH, '//div[contains(@data-tooltip,"Enviar") or contains(@data-tooltip,"Send")]'))
            send_btn.click()
        except Exception:
            # atalho teclado: Ctrl+Enter
            body = wait_css(driver, 'body')
            body.send_keys(Keys.CONTROL, Keys.ENTER)

        # pequena espera para sair do compose
        time.sleep(3)
        print("[OK] E-mail enviado via web.")
    finally:
        # Se quiser manter janela aberta para conferência, comente o quit()
        driver.quit()


def main():
    cfg = load_cfg()
    ensure_paths(cfg)

    # Relatório
    df = carregar_df(cfg["CSV_PATH"])
    k = calcular_kpis(df)
    ctx = {
        **k,
        "data_referencia": datetime.now().strftime("%d/%m/%Y %H:%M"),
        "organizacao": cfg["ORGANIZACAO"],
    }
    html = render_html(cfg["TEMPLATE_PATH"], ctx)
    html_path = salvar_html(html, cfg["REPORTS_DIR"])
    print(f"[INFO] HTML salvo em: {html_path}")

    # Enviar via web (Selenium)
    enviar_email_via_gmail(cfg, html, anexar=html_path)


if __name__ == "__main__":
    main()

```

---
# ▶️ Como executar


```bash
# 1) (recomendado) criar venv
python -m venv .venv
# Windows
.venv\Scripts\activate
# Linux/Mac
# source .venv/bin/activate

# 2) instalar deps
pip install -r requirements.txt

# 3) conferir .env, data/vendas.csv e templates/email.html.j2

# 4) rodar
python send_report_selenium.py

```
